In [1]:
! pip install "unsloth[colab] @ git+https://github.com/unslothai/unsloth.git" transformers trl wandb

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-6s4tehsx/unsloth_3016d6d50d0743ac97ef412c3015edee
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-6s4tehsx/unsloth_3016d6d50d0743ac97ef412c3015edee
  Resolved https://github.com/unslothai/unsloth.git to commit 172219e3e76e5508e97da3e5e281597a4246dcb7
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.8/211.8 MB 7.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.4/102.4 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 13.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.2/245.2 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━

In [2]:
! pip install bitsandbytes

In [3]:
from unsloth import FastLanguageModel 
from trl import SFTTrainer 
from transformers import TrainingArguments
from datasets import load_dataset
import torch

/opt/conda/lib/python3.10/site-packages/unsloth/__init__.py:80: UserWarning: Unsloth: Running `ldconfig /usr/lib64-nvidia` to link CUDA.
  warnings.warn(


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


2024-06-05 20:53:15.829993: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-05 20:53:15.830127: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-05 20:53:15.973850: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
dataset = load_dataset("lilacai/glaive-function-calling-v2-sharegpt", split='train')


Generating train split:   0%|          | 0/112960 [00:00<?, ? examples/s]

In [5]:
model, tokenizer = FastLanguageModel.from_pretrained(
                                model_name = "unsloth/gemma-1.1-2b-it-bnb-4bit",
                                max_seq_length = 512,
                                load_in_4bit = True,
                                dtype = None
                                )

config.json:   0%|          | 0.00/1.13k [00:00<?, ?B/s]

==((====))==  Unsloth: Fast Gemma patching release 2024.5
   \\   /|    GPU: Tesla P100-PCIE-16GB. Max memory: 15.895 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.1.0+cu121. CUDA = 6.0. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. Xformers = 0.0.22.post7. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


model.safetensors:   0%|          | 0.00/2.07G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/40.6k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

In [6]:
model = FastLanguageModel.get_peft_model(
    
    model,
    r = 16,
    lora_alpha = 16,
    lora_dropout = 0,
    bias = 'none',
    target_modules = ['q_proj', 'k_proj', 'v_proj', 'gate_proj', 'up_proj', 'down_proj'],
    use_gradient_checkpointing = True,
    max_seq_length = 512,
    use_rslora = False,
    loftq_config = False,
    )

Not an error, but Unsloth cannot patch O projection layer with our manual autograd engine since either LoRA adapters
are not enabled or a bias term (like in Qwen) is used.
Unsloth 2024.5 patched 18 layers with 18 QKV layers, 0 O layers and 18 MLP layers.


In [13]:
model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): GemmaForCausalLM(
      (model): GemmaModel(
        (embed_tokens): Embedding(256000, 2048)
        (layers): ModuleList(
          (0-17): 18 x GemmaDecoderLayer(
            (self_attn): GemmaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=2048, out_features=2048, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2048, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=2048, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=2048, o

In [7]:
from unsloth.chat_templates import get_chat_template 

tokenizer = get_chat_template(
tokenizer,
chat_template = 'chatml',
mapping={"role" : "from", "content" : "value", "user" : "human", "assistant" : "gpt"}, 
    map_eos_token=True)

In [8]:
def formatting_func(examples):
    convs = examples['conversations']
    texts = [tokenizer.apply_chat_template(conv, tokenize=False, add_generation_prompt = False) for conv in convs]
    return {'text' : texts}

dataset = dataset.map(formatting_func, batched = True)

Map:   0%|          | 0/112960 [00:00<?, ? examples/s]

In [9]:
print(dataset['text'][0])

<bos><|im_start|>system
You are a helpful assistant with access to the following functions. Use them if required -
{
    "name": "calculate_median",
    "description": "Calculate the median of a list of numbers",
    "parameters": {
        "type": "object",
        "properties": {
            "numbers": {
                "type": "array",
                "items": {
                    "type": "number"
                },
                "description": "The list of numbers"
            }
        },
        "required": [
            "numbers"
        ]
    }
}
<|im_end|>
<|im_start|>user
Hi, I have a list of numbers and I need to find the median. The numbers are 5, 2, 9, 1, 7, 4, 6, 3, 8.<|im_end|>
<|im_start|>assistant
<functioncall> {"name": "calculate_median", "arguments": '{"numbers": [5, 2, 9, 1, 7, 4, 6, 3, 8]}'} <|endoftext|><|im_end|>
<|im_start|>system
{"median": 5}<|im_end|>
<|im_start|>assistant
The median of your list of numbers is 5. <|endoftext|><|im_end|>



In [10]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
wandb_key = user_secrets.get_secret("wandb_key")
hf_api = user_secrets.get_secret("HF_API")

In [11]:
import wandb
wandb.login(key=wandb_key)

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [12]:
from huggingface_hub import login
login(token = hf_api)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [13]:
trainer = SFTTrainer(
    model,
    train_dataset = dataset,
    dataset_text_field = 'text',
    max_seq_length = 512,
    packing = False,
    dataset_num_proc = 2,
    tokenizer = tokenizer,
    args = TrainingArguments(
            per_device_train_batch_size = 7,
            gradient_accumulation_steps = 1,
        warmup_steps = 10,
        max_steps  = 1000,
        learning_rate = 2e-5,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 50,
        output_dir = 'unsloth-gemma-glaive-func-calling',
        optim = 'adamw_8bit',
        num_train_epochs = 3,
        push_to_hub = True,
        report_to = 'wandb'
            )
    )

/opt/conda/lib/python3.10/site-packages/multiprocess/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Map (num_proc=2):   0%|          | 0/112960 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [14]:
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 112,960 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 7 | Gradient Accumulation steps = 1
\        /    Total batch size = 7 | Total steps = 1,000
 "-____-"     Number of trainable parameters = 18,432,000
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Currently logged in as: amaithi-sam (amaithi). Use `wandb login --relogin` to force relogin


Step,Training Loss
50,2.166200
100,0.000000
150,0.000000
200,0.000000
250,0.000000
300,0.000000
350,0.000000
400,0.000000
450,0.000000
500,0.000000


TrainOutput(global_step=1000, training_loss=0.1083125, metrics={'train_runtime': 7361.1927, 'train_samples_per_second': 0.951, 'train_steps_per_second': 0.136, 'total_flos': 5.418984611286221e+16, 'train_loss': 0.1083125, 'epoch': 0.061965547155781384})